# Software gestionale per negozio di prodotti vegani


**RICHIESTA:** BioMarket s.a.s richiede un piccolo software gestionale per la loro nuova bottega in via Tan 6. Tale software deve avere le seguenti funzionalità:


*   Registrare nuovi prodotti, con nome, quantità, prezzo di vendita e prezzo d'acquisto.
*   Elencare tutti i prodotti presenti.
*   Registrare le vendite effettuate.
*   Mostrare i profitti lordi e netti.
*   Mostrare un meno di aiuto con tutti i comandi disponibili.

Il software è testuale, quindi utilizzabile da riga di comando.






#Sviluppo del software by Samuele Di Martino

Innanzitutto, predispongo due funzioni che hanno lo scopo di rendere il programma persistente. \
Nello specifico, con l'ausilio di un file json, posso tenere traccia e, ove necessario, modificare l'inventario del magazzino.\
Il programma, solo per il primo utilizzo, creerà da zero un dizionario con la seguente struttura:


1.   chiave *products* ----> valore costituito da una lista con dizionari come elementi. Ogni dizionario al suo interno è un prodotto con i relativi prezzi (di acquisto e di vendita).
2.   chiave *sales* --->come valore ha una lista di dizionari. Ciascuno di essi contiene il dettaglio di una vendita ed il profitto totale tratto da essa.
3.   chiave *gross_profit*---> come valore ha un float che rappresenta il profitto totale lordo dello store.
4.   chiave *net_profit* ---> come valore ha, invece, un float che rappresenta il profitto totale netto, ottenuto sottraendo al profitto lordo i costi di acquisto dei vari prodotti.

Per qualsiasi comando (valido) digitato dall'utente, il file json relativo all'inventario si modificherà in automatico, di pari alle modifiche del dizionario di partenza.


In [ ]:
import json

file_name = "inventory.json"

def load_inventory():
  """
  Function to load, or create, the inventory into the software
  """
  try:
    with open(file_name, "r") as inventory:
      return json.load(inventory)
  except FileNotFoundError:
    return {"products": [], "sales":[], "gross_profit": .0, "net_profit": .0}


def refresh_inventory(data):
  """
  Function to upload the inventory file with the changes made by the user.
  It modifies the file.
  """
  with open(file_name, "w") as inventory:
    json.dump(data, inventory, indent=3, ensure_ascii=False)



Ora, predispongo le funzioni per ciascun comando utilizzabile dall'utente che usufruirà del programma.\
I comandi ammessi, ciascuno con la propria funzionalità, sono:

*   aiuto -> funzione "help_me"
*   aggiungi -> funzione "add_product"
*   elenca -> funzione "show_inventory"
*   vendita -> funzione "register_sale"
*   profitti -> funzione "profits"

Le funzione sono definite come di seguito:

In [ ]:
#### help_me

def help_me():
  """
  When the user asks for help this function
  will show the list of the available commands
  """
  print("I comandi disponibili sono i seguenti:")
  print("aggiungi: aggiungi un prodotto al magazzino")
  print("elenca: elenca i prodotti in magazzino")
  print("vendita: registra una vendita effettuata")
  print("profitti: mostra i profitti totali")
  print("aiuto: mostra i possibili comandi")
  print("chiudi: esci dal programma")


#### is_aplha_with_space
# This function should be removed here and from the "add_product"
# and "register_sale" functions in case the store's products require
# names with special characters and/or numbers.

def is_aplha_with_space(name):
  """
  This function is used to check if the product name contains
  special characters or numbers.
  If such characters are not present, the function will return "True"
  """
  return all(part.isalpha() for part in name.split())



#### add_product

def add_product(data):
  """
  Function to add a product (with its quantity) to the inventory
  """
  product_name = input("Nome prodotto: ").strip().lower()
  # Checking if the name is empty or contains special characters or numbers
  if not product_name or not is_aplha_with_space(product_name):
    print("Errore: il nome prodotto inserito non è valido")
    print("Suggerimento: esso deve contenere solo lettere e spazi, " +
          "e non essere vuoto"
          )
    return

  # Inserting the quantity
  try:
    product_quantity = int(input("Quantità: "))
    assert(product_quantity>0), "Errore: la quantità deve essere un numero intero positivo"
  except ValueError:
    print("Errore: inserisci dei numeri validi per le quantità!")
    return
  except AssertionError as error:
    print(error)
    return
  # Checking the existence of the product into the inventory
  for item in data["products"]:
    if product_name==item["name"]:
      item["quantity"]+=product_quantity
      print(f"AGGIUNTO: {product_quantity} X {product_name}")
      refresh_inventory(data)
      return
  # If the product is not available, the program takes the prices in input
  try:
    purchase_price = float(input("Prezzo di acquisto: "))
    sale_price = float(input("Prezzo di vendita: "))
    if sale_price < purchase_price:
      print("Errore: il prezzo di vendita non può essere inferiore al prezzo "+
      "di acquisto")
      return
  except ValueError:
    print("Errore: inserisci dei numeri validi per i prezzi!")
    return

  # Registering the new product (only if not available)
  new_product = {
      "name": product_name,
      "quantity": product_quantity,
      "purchase_price": purchase_price,
      "sale_price": sale_price
  }
  data["products"].append(new_product)
  print(f"AGGIUNTO: {product_quantity} X {product_name}")
  refresh_inventory(data)



#### show_inventory

def show_inventory(data):
  """
  Function used to show the inventory to the user
  """
  print(f"{'PRODOTTO':<20} {'QUANTITÀ':<10} {'PREZZO':<10}")
  for product in data["products"]:
    print(f"{product['name']:<20} {product['quantity']:<12}"+
         f"€{product['sale_price']:<10.2f}"
         )



#### register_sale

def register_sale(data):
  """
  Used to register the sales of the store
  """
  order_revenue = .0
  net_revenue = .0 # to calculate the net profit
  sale_details = []

  while True:
    product_name = input("Nome prodotto: ").strip().lower()
    # Checking if the name is empty or contains special characters or numbers
    if not product_name or not is_aplha_with_space(product_name):
      print("Errore: il nome prodotto inserito non è valido")
      print("Suggerimento: esso deve contenere solo lettere e spazi, " +
            "e non essere vuoto"
            )
      continue

    # Inserting the quantity
    try:
      product_quantity = int(input("Quantità: "))
      assert(product_quantity>0), "Errore: la quantità deve essere un numero intero positivo"
    except ValueError:
      print("Errore: inserisci dei numeri validi per le quantità!")
      continue
    except AssertionError as error:
      print(error)
      continue


    # Checking the existence of the product in the inventory
    product_found = False
    for product in data["products"]:
      if product_name==product["name"]:
        product_found = True
        # Checking the quantity available
        if product_quantity<=product["quantity"]:
          product["quantity"]-=product_quantity
          sale_details.append(f"{product_quantity} X {product_name}: "+
                              f"€{product['sale_price']:.2f}"
                              )
          order_revenue+=product["sale_price"]*product_quantity
          net_revenue+=product_quantity*(product["sale_price"]-
                                        product["purchase_price"])
        else:
          print(f"Errore: La quantità di {product_name} disponibile "+
                "è inferiore a quella selezionata"
                )
        break

    if not product_found:
      print(f"Errore: al momento {product_name} "+
           "non risulta essere presente nell'inventario"
           )

    # Still in the while loop
    # Checking if the user wants to add another product
    another_product = input("Aggiungere un altro prodotto? (si/no): ").lower()
    if another_product!="si":
      break

  # Only if the user has selected at least one available product...
  # with the correct quantities, the program will record the sale
  if sale_details!=[]:
    data["sales"].append({
        "details": sale_details,
        "revenue": round(order_revenue, 2)
        })
    data["gross_profit"]+=order_revenue
    data["net_profit"]+=net_revenue
    #rounding the profits
    data["gross_profit"] = round(data["gross_profit"], 2)
    data["net_profit"] = round(data["net_profit"], 2)

  # Now the sale is registered
    print("VENDITA REGISTRATA")
    for detail in sale_details:
      print(f"- {detail}")
    print(f"Totale: €{order_revenue:.2f}")
    refresh_inventory(data)
  else:
    return



#### profits

def profits(data):
  """
  Function that shows the profits of the store, gross and net
  """
  print(f"Profitto: lordo=€{data['gross_profit']:.2f} "+
       f"netto=€{data['net_profit']:.2f}"
       )


Adesso ho a disposizione tutte le funzioni necessarie per il corretto funzionamento del programma.\
 Posso procedere, dunque, ad assemblarle in un'unica funzione principale che gestirà tutto il programma.

In [ ]:
def main():
  data = load_inventory()

  while True:
  # It exits, and close the program, only if the user types the appropriate command
    user_command = input("Inserisci un comando: ").strip().lower()

    if user_command=="aiuto":
      help_me()
    elif user_command=="aggiungi":
      add_product(data)
    elif user_command=="elenca":
      show_inventory(data)
    elif user_command=="vendita":
      register_sale(data)
    elif user_command=="profitti":
      profits(data)
    elif user_command=="chiudi":
      print("Bye bye")
      break
    else:
      print("Comando non valido")
      help_me()

Infine, definisco il punto di ingresso del programma.\
In questo modo riesco a rendere lo script **riutilizzabile**, sia da terminale che importandolo altrove.

In [ ]:
# This is the entry point of the program
if __name__=="__main__":
  main()